# Langchain与知识图谱结合问答

针对图数据库的自动问答：所谓的自动，就是不需要人工定义模板，让大模型自动生成Cypher语句，然后查询数据库，得到答案，要用的链是GraphCypherQAChain。

In [1]:
%pip install neo4j

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/Note: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/301.7 kB ? eta -:--:--
     ----------- --------------------------- 92.2/301.7 kB 2.6 MB/s eta 0:00:01
     -------------------------------------  297.0/301.7 kB 3.7 MB/s eta 0:00:01
     -------------------------------------- 301.7/301.7 kB 3.7 MB/s eta 0:00:00


此外，还需要安装apoc插件，将neo4j/labs/apoc-5.25.1-core -> plugins/apoc-5.25.1-core 即可。

之后修改配置项，

```conf
# 修改conf/neo4j.conf
server.directories.plugins=plugins
dbms.security.procedures.allowlist=*
dbms.security.procedures.unrestricted=*
```

最后重启neo4j验证：

```cypher
return apoc.version();
```

In [2]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model='qwen2:7b', temperature=0)

In [4]:
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain

graph = Neo4jGraph(
    'neo4j://localhost:7687',
    'neo4j',     # 用户名
    '***', # 密码
    'test'    # 数据库
)

chain = GraphCypherQAChain.from_llm(
    llm = llm,
    graph = graph,
    verbose = True
)

result = chain.run('过敏性鼻炎能治好吗？')
result



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "过敏性鼻炎"})-[:DISEASE_CUREWAY]->(c:Cureway) RETURN c.name LIMIT 1
Full Context:
[{'c.name': '支持性治疗'}]

> Finished chain.


'支持性治疗是管理过敏性鼻炎的有效方法之一。通过这种方式，可以减轻症状并提高生活质量。然而，完全治愈过敏性鼻炎可能具有挑战性，因为这通常涉及到对环境因素的持续控制和避免接触已知的过敏原。此外，个人的生活方式、饮食习惯以及使用药物等综合措施也起着重要作用。在某些情况下，通过这些方法可以显著改善症状，使得生活质量得到极大提升。'